In [1]:
import pickle
import pandas as pd
import numpy as np

In [2]:
model = pickle.load(open('./models/xgboost_regressor_1.pkl', 'rb'))

In [3]:
# preprocessors = pickle.load(open('preprocessors.pkl', 'rb'))
preprocessors = pickle.load(open('preprocessors_for_grouped_industry.pkl', 'rb'))

In [4]:
def split_loan_range(df):
    df['Loan_min'] = df['LoanRange'].map({
        '$16300-27300': 16300, '$7964-16300': 7964, '$0-7964': 0, '$56313-150000':56313,
           '$27300-56313': 27300, '$150,000-350,000': 150000, '$350,000-1 million':350000,
           '$1-2 million': 1000000, '$2-5 million': 2000000, '$5-10 million': 5000000
    })

    df['Loan_max'] = df['LoanRange'].map({
        '$16300-27300': 27300, '$7964-16300': 16300, '$0-7964': 7964, '$56313-150000':150000,
           '$27300-56313': 56313, '$150,000-350,000': 350000, '$350,000-1 million':1000000,
           '$1-2 million': 2000000, '$2-5 million': 5000000, '$5-10 million': 10000000
    })
    df = df.drop(columns=['LoanRange'], axis=1)
    return df

In [10]:
test_data = pd.read_csv('./2020 PPP Dataset/PPP Test ALL.csv')
test_data.describe()

,Unnamed: 0,NAICSCode,Zip,Index
count,350000.000000,350000.000000,349998.000000,3.500000e+05
mean,174999.500000,529273.102820,51496.690767,2.266688e+06
std,101036.441446,179241.740844,29547.010964,1.319605e+06
min,0.000000,111110.000000,1001.000000,3.000000e+00
25%,87499.750000,445110.000000,28572.000000,1.126375e+06
50%,174999.500000,541211.000000,49417.000000,2.260489e+06
75%,262499.250000,621610.000000,78154.000000,3.399558e+06
max,349999.000000,928120.000000,99950.000000,4.561256e+06


In [6]:
test_data.shape

(350000, 16)

In [11]:
test_data['BusinessType'] = test_data['BusinessType'].fillna('Other_BT')
test_data['City'] = test_data['City'].fillna('Other_City')
test_data['Zip'] = test_data['Zip'].fillna('Other').astype(str)
test_data['NAICSCode'] = test_data['NAICSCode'].astype(str)

In [12]:
test_df = test_data.copy()

In [13]:
test_df = split_loan_range(test_df)

In [14]:
test_df = test_df[preprocessors['selected_features']]
test_df.head()

,DateApproved,Gender,RaceEthnicity,Veteran,Industry,Loan_min,Loan_max
0,04/30/2020,Unanswered,Unanswered,Unanswered,Offices of Real Estate Agents and Brokers,0,7964
1,04/28/2020,Male Owned,Unanswered,Unanswered,Drycleaning and Laundry Services (except Coin-...,7964,16300
2,04/13/2020,Unanswered,Unanswered,Unanswered,Other Direct Selling Establishments,27300,56313
3,05/08/2020,Unanswered,Unanswered,Unanswered,Snack and Nonalcoholic Beverage Bars,0,7964
4,04/27/2020,Unanswered,Unanswered,Unanswered,Beauty Salons,16300,27300


In [15]:
test_df.describe(include='all')

,DateApproved,Gender,RaceEthnicity,Veteran,Industry,Loan_min,Loan_max
count,350000,350000,350000,350000,350000,3.500000e+05,3.500000e+05
unique,79,3,8,3,1056,NaN,NaN
top,04/28/2020,Unanswered,Unanswered,Unanswered,Full-Service Restaurants,NaN,NaN
freq,32934,268290,309722,294824,12840,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,7.210647e+04,1.734960e+05
std,NaN,NaN,NaN,NaN,NaN,2.447741e+05,5.491557e+05
min,NaN,NaN,NaN,NaN,NaN,0.000000e+00,7.964000e+03
25%,NaN,NaN,NaN,NaN,NaN,7.964000e+03,1.630000e+04
50%,NaN,NaN,NaN,NaN,NaN,1.630000e+04,2.730000e+04
75%,NaN,NaN,NaN,NaN,NaN,5.631300e+04,1.500000e+05


In [16]:
test_df['RaceEthnicity'].value_counts()

Unanswered                          309722
White                                31776
Asian                                 4228
Hispanic                              2837
Black or African American             1224
American Indian or Alaska Native       211
Eskimo & Aleut                           1
Puerto Rican                             1
Name: RaceEthnicity, dtype: int64

In [17]:
industry_groups = preprocessors['industry_groups']
test_df['Industry'] = test_df['Industry'].apply(lambda s: s.strip())
for group in iter(industry_groups):
    for industry in industry_groups[group]:
        test_df['Industry'] = test_df['Industry'].replace(industry, group)

print(test_df['Industry'].unique().shape)

(109,)


In [18]:
test_df['Industry'].unique()

array(['Industry_Group_0', 'Industry_Group_38', 'Industry_Group_48',
       'Industry_Group_39', 'Industry_Group_45', 'Industry_Group_12',
       'Industry_Group_67', 'Industry_Group_22', 'Industry_Group_25',
       'Industry_Group_23', 'Industry_Group_103', 'Industry_Group_1',
       'Industry_Group_34', 'Industry_Group_52', 'Industry_Group_44',
       'Industry_Group_33', 'Industry_Group_73', 'Industry_Group_46',
       'Industry_Group_54', 'Industry_Group_6', 'Industry_Group_66',
       'Industry_Group_70', 'Industry_Group_90', 'Industry_Group_96',
       'Industry_Group_35', 'Industry_Group_51', 'Industry_Group_2',
       'Industry_Group_71', 'Industry_Group_41', 'Industry_Group_7',
       'Industry_Group_17', 'Industry_Group_82', 'Industry_Group_3',
       'Industry_Group_87', 'Industry_Group_42', 'Industry_Group_57',
       'Industry_Group_91', 'Industry_Group_4', 'Industry_Group_18',
       'Industry_Group_69', 'Industry_Group_80', 'Industry_Group_84',
       'Industry_Group_14'

In [19]:
test_encoded_scaled = preprocessors['loan_scaler'].transform(test_df[['Loan_min', 'Loan_max']])

for feature in preprocessors['selected_features'][:-2]:
    encoder = preprocessors['feature_encoder_scaler'][feature]['encoder']
    scaler = preprocessors['feature_encoder_scaler'][feature]['scaler']
    encoded_feature = encoder.transform(test_df[feature].values.reshape(-1,1))
    scaled_feature = scaler.transform(encoded_feature.toarray())
#     if feature == 'Industry':
#         scaled_feature = preprocessors['industry_pca'].transform(scaled_feature)
#         continue
    
    test_encoded_scaled = np.hstack((scaled_feature, test_encoded_scaled))

In [20]:
test_encoded_scaled.shape

(350000, 200)

In [21]:
pred = model.predict(test_encoded_scaled)
pred = np.floor(pred).astype(int)
pred[pred < 0] = 0

In [22]:
pred.shape

(350000,)

In [23]:
submission = pd.DataFrame(columns=['Index', 'JobsRetained'])
submission['Index'] = test_data['Index'].copy()
submission['JobsRetained'] = pred

In [24]:
submission

,Index,JobsRetained
0,1705962,1
1,770689,1
2,4016874,5
3,2306105,1
4,4517956,2
...,...,...
349995,1967756,5
349996,1831491,1
349997,1778539,2
349998,2602834,4


In [25]:
submission.set_index('Index',inplace=True)

In [26]:
submission.to_csv('./Submissions/Animesh_Submission_13.csv')